# Integrações com o CulinaryDB - Equipe GPALT (Analistas de Cardápios)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db


# Importando Tabelas dos dados originais do CulinaryDB

### Associando ingredientes compostos

As tabelas a seguir podem ser importadas a partir dos dados do notebook anterior:

In [2]:
DROP TABLE IF EXISTS AssociacaoComFooDB;
DROP TABLE IF EXISTS ComposicaoIngredientes;

CREATE TABLE AssociacaoComFooDB (
  Nome VARCHAR(100),
  FooDB VARCHAR(100),
  PRIMARY KEY (Nome)
) AS SELECT
  nome_ingrediente, nome_foodb
FROM CSVREAD('../data/external/associacaofinal.csv');

CREATE TABLE ComposicaoIngredientes (
  IngredienteComposto VARCHAR(100),
  IngredienteOriginal VARCHAR(100),
  PRIMARY KEY (IngredienteComposto, IngredienteOriginal)
) AS SELECT
  IngredienteComposto,IngredienteOriginal
FROM CSVREAD('../data/interim/culinarydb/associacaocompostos.csv');


Visualizando os dados de composição:

In [3]:
SELECT * FROM ComposicaoIngredientes LIMIT 10;


Associando os ingredientes que formam os ingredientes compostos com os nomes no FooDB usando uma VIEW:

In [4]:
DROP VIEW IF EXISTS IngredienteComposto;

CREATE VIEW IngredienteComposto AS
SELECT DISTINCT CI.IngredienteComposto, A.FooDB IngredienteOriginal
FROM AssociacaoComFooDB A, ComposicaoIngredientes CI
WHERE CI.IngredienteOriginal = A.Nome
ORDER BY CI.IngredienteComposto;


Visualizando os dados:

In [5]:
SELECT * FROM IngredienteComposto LIMIT 10;


Esses dados ainda vão sofrer alterações na construção final, pois ainda é necessário verificar se esses ingredientes já existem na base do FooDB.

Exportando:

In [6]:
CALL CSVWRITE('../data/interim/culinarydb/ingredientecomposto.csv',
              'SELECT * FROM IngredienteComposto');


463

### Ingredientes nas Receitas

Agora, é preciso atualizar o nome dos ingredientes nas receitas de acordo com o modelo proposto. Criando as tabelas necessárias nessa etapa:

In [7]:
DROP TABLE IF EXISTS IngredientesSimples;
DROP TABLE IF EXISTS IngredientesCompostos;
DROP TABLE IF EXISTS Receitas;
DROP TABLE IF EXISTS IngredientesReceitas;

CREATE TABLE IngredientesSimples (
  NomeSimples VARCHAR(100),
  IDSimples INT,
  PRIMARY KEY (IDSimples)
) AS SELECT
  Nome, ID
FROM CSVREAD('../data/interim/culinarydb/nomesingredients.csv');

CREATE TABLE IngredientesCompostos (
  NomeComposto VARCHAR(100),
  IDComposto INT,
  PRIMARY KEY (IDComposto)
) AS SELECT
  Compound_Ingredient_Name, entity_id
FROM CSVREAD('../data/interim/culinarydb/dadoscompostos.csv');

CREATE TABLE Receitas (
  IDReceita INT,
  NomeReceita VARCHAR(200),
  BancoOriginal VARCHAR(20),
  Origem VARCHAR(30),
  PRIMARY KEY (IDReceita)
) AS SELECT
  IDReceita, Nome, BancoOriginal, Origem
FROM CSVREAD('../data/interim/culinarydb/receitas.csv');

CREATE TABLE IngredientesReceitas (
  IDAssociacao INT,
  Receita INT,
  Quantidade DECIMAL(10,3),
  Unidade VARCHAR(30),
  Ingrediente INT,
  PRIMARY KEY (IDAssociacao)
) AS SELECT
  id_associacao,recipe_id,qtd,unit,ingredient
FROM CSVREAD('../data/interim/culinarydb/ingredientesnasreceitas.csv');


Visualizando as tabelas:

In [8]:
SELECT * FROM IngredientesSimples LIMIT 10;


In [9]:
SELECT * FROM IngredientesCompostos LIMIT 10;


In [10]:
SELECT * FROM Receitas LIMIT 10;


In [11]:
SELECT * FROM IngredientesReceitas LIMIT 10;


Criando uma VIEW com todos os ingredientes da base juntos:

In [12]:
DROP VIEW IF EXISTS IngredientesFinais;

CREATE VIEW IngredientesFinais AS
(SELECT * FROM IngredientesSimples
UNION
SELECT * FROM IngredientesCompostos);


Visualizando a VIEW:

In [13]:
SELECT * FROM IngredientesFinais LIMIT 10;


Alterando a unidade para desconhecida se não houver informação sobre a quantidade do ingrediente na receita:

In [14]:
UPDATE IngredientesReceitas
SET Unidade = 'desconhecida'
WHERE Quantidade = 0;


Segundo o modelo proposto, podemos criar a seguinte VIEW representando a interação entre receitas e ingredientes. Os ingredientes podem ser agrupados e as quantidades somadas caso haja repetição:

In [15]:
DROP VIEW IF EXISTS RelacaoFinal;

CREATE VIEW RelacaoFinal AS
SELECT R.IDReceita, R.BancoOriginal, A.FooDB, SUM(IR.Quantidade) Quantidade, IR.Unidade
FROM Receitas R, IngredientesFinais I, IngredientesReceitas IR, AssociacaoComFooDB A
WHERE R.IDReceita = IR.Receita AND I.IDSimples = IR.Ingrediente AND I.NomeSimples = A.Nome
GROUP BY R.IDReceita, R.BancoOriginal, A.FooDB, IR.Unidade
ORDER BY R.IDReceita, A.FooDB;


Visualizando:

In [16]:
SELECT * FROM RelacaoFinal LIMIT 10;


Exportando:

In [17]:
CALL CSVWRITE('../data/interim/culinarydb/receitasfinais.csv',
              'SELECT * FROM RelacaoFinal');


383045

### Origem das receitas

Separando os lugares de origem das receitas e selecionando as únicas:

In [18]:
DROP VIEW IF EXISTS Regioes;

CREATE VIEW Regioes AS
    SELECT DISTINCT Origem
    FROM Receitas;


Visualizando:

In [19]:
SELECT * FROM Regioes LIMIT 10;


Exportando:

In [20]:
CALL CSVWRITE('../data/interim/culinarydb/origens.csv',
              'SELECT * FROM Regioes');


26